In [ ]:
import pandas as pd
import os

# SCENIC steps

### STEP 1: Gene regulatory network inference, and generation of co-expression modules

In [ ]:
cluster = [
    't_CD4-CXCL13', 't_CD4_Tcm-LMNA', 't_CD4_Tn-LEF1',
       't_CD4_Treg-FOXP3', 't_CD8_MAIT-KLRB1',
           't_CD8_Tem-GZMK', 't_CD8_Trm-ZNF683', 't_Tact-XIST',
    't_Bmem-CD27','t_pB-IGHG1','t_macro-MMP9'
]

In [ ]:
# transcription factors list
f_tfs = "./data/allTFs_hg38.txt" # human

In [ ]:
for subcluster in cluster:
    # gene expression matrix directory
    f_loom_path_scenic = './data/' + subcluster + '_scenic.loom'
    os.mkdir('./output/'+subcluster)
    !/data/hour/miniconda3/bin/pyscenic grn {f_loom_path_scenic} {f_tfs} -o ./output/{subcluster}/adj.csv --num_workers 20
    

    

In [ ]:
f_db_names = './data/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather'
# motif databases
f_motif_path = './data/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl'

In [ ]:
for subcluster in cluster:
    f_loom_path_scenic = './data/' + subcluster + '_scenic.loom'
    !/data/hour/miniconda3/bin/pyscenic ctx ./output/{subcluster}/adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output ./output/{subcluster}/reg.csv \
    --mask_dropouts \
    --num_workers 20


In [ ]:
for subcluster in cluster:
    f_loom_path_scenic = './data/' + subcluster + '_scenic.loom'
    
    !/data/hour/miniconda3/bin/pyscenic aucell \
    {f_loom_path_scenic} \
    ./output/{subcluster}/reg.csv \
    --output {'./output/'+subcluster+'/pyscenic_output.loom'} \
    --num_workers 1

In [ ]:
# gene expression matrix directory
f_loom_path_scenic = './data/' + subcluster + '_scenic.loom'

In [ ]:
!/data/hour/miniconda3/bin/pyscenic grn {f_loom_path_scenic} {f_tfs} -o ./output/t_CD4_Tn-LEF1/adj.csv --num_workers 40

##### read in the adjacencies matrix:

In [ ]:
adjacencies = pd.read_csv("./output/Naive/adj.csv", index_col=False, sep=',')

In [ ]:
adjacencies.head()

### STEP 2-3: Regulon prediction aka cisTarget from CLI

In [ ]:
f_db_names = './data/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather'
# motif databases
f_motif_path = './data/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl'

In [ ]:
!/data/hour/miniconda3/bin/pyscenic ctx ./output/Naive/adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output ./output/Naive/reg.csv \
    --mask_dropouts \
    --num_workers 40

### STEP 4: Cellular enrichment (aka AUCell) from CLI

In [ ]:
!/data/hour/miniconda3/bin/pyscenic aucell \
    {f_loom_path_scenic} \
    ./output/t_mDC-LAMP3/reg.csv \
    --output {'./output/t_mDC-LAMP3/pyscenic_output.loom'} \
    --num_workers 40